# Cargamos las ejecuciones de la tarea anterior

## Eliminación de cabeceras.
Se ha realizado en la Tarea 3.

In [1]:
import os
path = os.getcwd() + "/Corpus-representacion"
# Obtener el listado
listado = os.listdir(path)
corpus_dir = [] # Guardamos el directorio para cada fichero
for elemento in listado:
    corpus_dir.append(path +'/'+elemento)

all_corpus_files = []
for dir in corpus_dir:
    corpus_files = os.listdir(dir)
    for corpus in corpus_files:
        all_corpus_files.append(dir +'/'+corpus)

In [2]:
listado

['talk.politics.mideast',
 'rec.autos',
 'comp.sys.mac.hardware',
 'rec.sport.hockey',
 'comp.sys.ibm.pc.hardware',
 'sci.electronics',
 'talk.politics.guns']

In [3]:
def read_file(ruta_fichero):
    """
    Lee el contenido de un fichero de texto y lo retorna como un string único.
    
    Args:
        ruta_fichero (str): Ruta al archivo de texto.
        
    Returns:
        str: Contenido completo del fichero como una única cadena.
    """
    try:
        with open(ruta_fichero, 'r', encoding='utf-8') as fichero:
            contenido = fichero.read()
        return contenido
    except FileNotFoundError:
        return "Error: El fichero no se encuentra."
    except Exception as e:
        return f"Error: {e}"

In [4]:
data = []
for corpus in all_corpus_files:
    for tema in listado:
        if tema in corpus:
            data.append((read_file(corpus), tema))
            break

In [5]:
import pandas as pd
df = pd.DataFrame(data, columns=['content', 'class'])
df.shape

(805, 2)

# Tokenización
La tokenización es el proceso de dividir un texto en unidades más pequeñas, conocidas como "tokens", que pueden ser palabras, frases o incluso caracteres. Este paso es crucial en el procesamiento de lenguaje natural (NLP) para convertir texto no estructurado en una forma que las máquinas puedan entender y analizar. La tokenización permite descomponer un documento en componentes manejables, como palabras individuales o sub-palabras, que posteriormente se utilizan en tareas como clasificación de texto, análisis de sentimientos, traducción automática, entre otras.

En el siguiente código, hemos estandarizado la entrada de las palabras eliminando aquellos términos que contienen símbolos como !, :, entre otros caracteres. Además, hemos convertido todos los caracteres a minúsculas, ya que esto mejorará el rendimiento de nuestro algoritmo. También hemos añadido un delimitador **@@@** al final de cada documento; en los pasos posteriores explicaremos el motivo de utilizar estos delimitadores.

In [6]:
df['content'][0]

'Message-ID: <FLAX.93Apr6125933@frej.teknikum.uu.se>References: <1993Mar30.142700.543@vms.huji.ac.il> <FLAX.93Apr3142133@frej.teknikum.uu.se><FLAX.93Apr5224449@frej.teknikum.uu.se><1993Apr5.221759.28472@thunder.mcrcim.mcgill.edu>NNTP-Posting-Host: frej.teknikum.uu.seIn-reply-to: hasan@McRCIM.McGill.EDU \'s message of Mon, 5 Apr 93 22:17:59 GMTIn article <1993Apr5.221759.28472@thunder.mcrcim.mcgill.edu> hasan@McRCIM.McGill.EDU  writes:[ stuff deleted ]|> I wrote:|> Are you calling names, or giving me a title? If the first, read your|> paragraph above, if not I accept the title, in order to let you get into the|> um, well, debate again.Hasan replies:I didnot know that "Master of wisdom" can be "name clling" too,unless you consider yourself deserve-less !Unless you are referring to someone else, you have in fact given me a nameI did not ask for, hence the term \'name calling\'.Hasan writes:|>    So what do you expect me to tell you to tell you, Master of Wsidom,|> \t\t\t\t\t\t\t       ^^^

In [19]:
import re
import nltk
from nltk.corpus import words as nltk_words
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag
# nltk.download('words')

valid_words = set(nltk_words.words())
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def convert_string_to_array(string_to_convert):
    # Dividir la cadena en palabras, eliminando espacios en blanco
    return [word for word in string_to_convert.split(' ') if word]

def remove_all_words_that_contain_numbers(list_of_words):
    # Verifica que cada elemento en la lista no contenga números
    return [word for word in list_of_words if not re.search(r'\d', word)]

def filter_word_by_lenght(list_of_words):
    exceptions = ['i', 'a']
    return [word for word in list_of_words if 2 <= len(word) <= 15 or word in exceptions]

def filter_stop_words(list_of_words):
    filtered_bag_of_words = []
    for word in list_of_words:
        if word not in stop_words:
            filtered_bag_of_words.append(word)
    return filtered_bag_of_words

def word_exists(list_of_words):
    return [word for word in list_of_words if word in valid_words or word == '@@@']

def array_to_string(list_of_words):
    return ' '.join(list_of_words)

# Función para mapear tipos gramaticales
def get_wordnet_pos(word):
    tag = pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def apply_lemmatizer(list_of_words):
    return [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in list_of_words]

def preproces(data):
    preproces_data = convert_string_to_array(data)
    preproces_data = remove_all_words_that_contain_numbers(preproces_data)
    preproces_data = filter_word_by_lenght(preproces_data)
    preproces_data = word_exists(preproces_data)
    preproces_data = filter_stop_words(preproces_data)
    preproces_data = apply_lemmatizer(preproces_data)
    return array_to_string(preproces_data)


test = "hello smaller macia2345 going gone asdfasdf45 god afternoon a i m"
test = convert_string_to_array(test)
test = remove_all_words_that_contain_numbers(test)
test = filter_word_by_lenght(test)
test = word_exists(test)
test = apply_lemmatizer(test)
test = array_to_string(test)
test

'hello small go go god afternoon a i'

In [8]:
df.head()

,content,class
0,Message-ID: <FLAX.93Apr6125933@frej.teknikum.u...,talk.politics.mideast
1,In article <1993Apr5.202800.27705@wam.umd.edu>...,talk.politics.mideast
2,DEPOSITION of VITALY NIKOLAYEVICH DANIELIAN [1...,talk.politics.mideast
3,Nntp-Posting-Host: saluda.columbiasc.ncr.comIn...,talk.politics.mideast
4,In article <iacovou.734063606@gurney> iacovou@...,talk.politics.mideast


In [9]:
for index, row in df.iterrows():
    df.loc[index, 'content'] = preproces(row['content'])

In [10]:
df.head()

,content,class
0,message of article stuff you calling or giving...,talk.politics.mideast
1,article Bonnie of article do who even believe ...,talk.politics.mideast
2,of School at people in town know what was happ...,talk.politics.mideast
3,article is getting are breaking the article in...,talk.politics.mideast
4,article even the most uncivilized of have of c...,talk.politics.mideast


In [24]:
# create new column for Word2Vec
df['tokens'] = df['content'].apply(lambda x: x.split())
df.head()

,content,class,tokens
0,message of article stuff you calling or giving...,talk.politics.mideast,"[message, of, article, stuff, you, calling, or..."
1,article Bonnie of article do who even believe ...,talk.politics.mideast,"[article, Bonnie, of, article, do, who, even, ..."
2,of School at people in town know what was happ...,talk.politics.mideast,"[of, School, at, people, in, town, know, what,..."
3,article is getting are breaking the article in...,talk.politics.mideast,"[article, is, getting, are, breaking, the, art..."
4,article even the most uncivilized of have of c...,talk.politics.mideast,"[article, even, the, most, uncivilized, of, ha..."


# Funciones de pesado TF y TF-IDF


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Suponiendo que tu DataFrame se llama 'df' y la columna de texto es 'content'
contents = df['content']

# Crear un objeto TfidfVectorizer
vectorizer = TfidfVectorizer()

# Ajustar y transformar los datos de texto a TF-IDF
tfidf_matrix = vectorizer.fit_transform(contents)

# tfidf_matrix es una matriz dispersa (sparse matrix). Si deseas convertirla a una matriz densa:
dense_matrix = tfidf_matrix.toarray()

# Opcional: Obtener los nombres de las características (palabras o términos únicos)
feature_names = vectorizer.get_feature_names_out()

# Mostrar resultados
print("TF-IDF Matrix Shape:", tfidf_matrix.shape)  # Dimensiones: (número de documentos, número de términos)
print("Feature Names:", feature_names)


TF-IDF Matrix Shape: (805, 5802)
Feature Names: ['abandoned' 'abiding' 'ability' ... 'zone' 'zucchini' 'zulu']


In [25]:
from gensim.models import Word2Vec

# Entrenar el modelo Word2Vec
model = Word2Vec(
    sentences=df['tokens'],  # Lista de listas de palabras
    vector_size=100,         # Dimensión de los vectores
    window=5,                # Contexto de palabras (tamaño de la ventana)
    min_count=2,             # Mínimo número de ocurrencias para considerar una palabra
    workers=4,               # Número de hilos (núcleos de CPU)
    sg=1                     # Skip-gram (1) o CBOW (0)
)

# Guardar el modelo entrenado (opcional)
# model.save("word2vec_model")

# Opcional: Cargar el modelo posteriormente
# model = Word2Vec.load("word2vec_model")

# Inspeccionar las palabras más similares a un término
similar_words = model.wv.most_similar("article", topn=5)
print("Palabras más similares a 'article':", similar_words)

# Obtener el vector de una palabra
vector = model.wv['article']
print("Vector para 'article':", vector)

Palabras más similares a 'article': [('Jason', 0.8123925924301147), ('am', 0.7839433550834656), ('message', 0.7601901888847351), ('read', 0.7472124099731445), ('about', 0.715707004070282)]
Vector para 'article': [-0.30588853  0.2768493  -0.15653813 -0.01074309 -0.02767886 -0.02225285
  0.19732454  0.21759912  0.03973274  0.12874821  0.27892548 -0.6338676
  0.17702127 -0.03644745  0.16711238 -0.11877959 -0.01292252 -0.02337271
 -0.0452549   0.09049419 -0.16342013 -0.24691194 -0.17946075 -0.11492638
  0.05154823  0.0882732  -0.22497827  0.10889766 -0.26830366 -0.3273254
 -0.2232684  -0.08670915  0.2755796  -0.11315802 -0.2654038   0.32977548
  0.28554755  0.21898916 -0.49614552  0.13591675 -0.1149618   0.17802273
  0.22116116 -0.36805755  0.41195393 -0.2840177   0.05711083 -0.17387007
  0.38688904  0.371374    0.27710453 -0.09631194 -0.16103676 -0.16869402
  0.04634953 -0.23770972  0.20871618 -0.10144551  0.41607937 -0.08494373
 -0.04631688 -0.08393058 -0.0280866   0.10491826 -0.08547763

# K-means
https://www.kaggle.com/code/aybukehamideak/clustering-text-documents-using-k-means